# **STEP1**

Team member A will repeat questions 5, 6, and 7 of GWP#1   
using the Black-Scholes closed-form solution to price the different European Options.   
For Q7 on vega, you can use Black-Scholes closed-form solution.

In [1]:
import numpy as np
from scipy.stats import norm
import pandas as pd

def BSprice_n_Greeks(S0, K, mu, sigma, T, t, opt='call', yield_rate=0):
    dt = T - t
    vol = sigma * np.sqrt(dt)
    d1 = ( np.log(S0/K) + (mu - yield_rate + 0.5 * sigma**2) * (dt)) / vol
    d2 = d1 - sigma * np.sqrt(dt)

    if opt == 'call':
        price = S0 * np.exp(-yield_rate * dt) * norm.cdf(d1) - K * np.exp(-mu * dt) * norm.cdf(d2)
        delta = norm.cdf(d1)
        vega = S0 * norm.pdf(d1) * np.sqrt(dt) / 100

    elif opt == 'put':
        price = K * np.exp(-mu * dt) * norm.cdf(-d2) - S0 * np.exp(-yield_rate * dt) * norm.cdf(-d1)
        delta = -norm.cdf(-d1)
        vega = S0 * norm.pdf(d1) * np.sqrt(dt) / 100
    
    df = pd.DataFrame({f"price_{opt}": price, f"delta_{opt}": delta, f"vega_{opt}": vega}, index=[0])

    return df

$S_0 = 100; r = 5\%; σ = 20\%; T=3months$

In [2]:
S0 = 100
K = S0 # ATM
r = 0.05
sigma = 0.2
T = 3/12
t = 0

In [3]:
np.random.seed(0)
df_call = BSprice_n_Greeks(S0=S0, K=S0, mu=r, sigma=sigma, \
                                    T=T, t=t, opt='call', yield_rate=0)
df_put = BSprice_n_Greeks(S0=S0, K=S0, mu=r, sigma=sigma, 
                                    T=T, t=t, opt='put', yield_rate=0)

**(1)**  
Black Scholes Model:  
$$
\begin{aligned}
d1 &= \frac{ln(\frac{S}{K})+(μ-\frac{σ^2}{2})(T-t)}{σ\sqrt{T-t}} \quad & (1) \\ 
d2 &= d1 - σ \sqrt{T-t} \quad & (2)     \\
C_{BS} &= S_0 N(d1) - K e^{-rT} N(d2) \quad & (3)  \\
C_{BS} &= K e^{-rT} N(-d2) - S_0 N(-d1) \quad & (4)  \\
\end{aligned}
$$

With Black Scholes Model, the steps are continuous. So we don't need to choose number of steps.

A brief description of the overall process as follows:
1. From providing information, we calculate components of the Black Scholes model
2. Plug the numbers into d1 and d2 as formula is mentioned (1) and (2) above
3. Calculate the value of the **European Call Option** by using the Black Scholes by replacing to (3)
4. Next, we could calculate the **European Put Option** by using Put-Call Partiy or (4)
5. Lastly, we could calculate the Greeks like delta, gamma, vega, ...

In [4]:
df_call

,price_call,delta_call,vega_call
0,4.614997,0.56946,0.19644


In [5]:
df_put

,price_put,delta_put,vega_put
0,3.372777,-0.43054,0.19644


The calculated price for the ATM European options using Black Scholes model is:
* European Call Option Price = **4.61**
* European Put Option Price = **3.37**

**(6)**  
* European Call Option Delta = **0.57**  
* European Put Option Delta = **-0.43**  

The delta of the European Call Option is positive, while the Delta of the European Put Option is negative. This is consistent with the general behaviour of the options with respect to underlying asset's price movements.

Delta primarily act as a proxy for the sensitivity of the option's price to small movements in the underlying asset's price. It essentially indicates how much the price of an option is expected to change for a $1 change in the price of the underlying asset.

The positive Delta for the European call option indicates that its value is expected to increase as the underlying asset's price increases. This aligns with the nature of a call option, which provides the right to buy underlying asset. Hence, when the asset's price goes up, the option becomes more valuable.

The negative Delta for the European put option indicates that its value is expected to increase as the underlying asset's price decreases. This is consistent with the nature of a put option, which provides the right to sell the underlying asset. Therefore, when the asset's price falls, the option gains value.

The signs of Delta for oth options are in line with their respective payoff structures. A call option benefits from upward movements in the underlying asset, while a put option benefits from downward movements . This fundamental difference is reflected in the positive Delta for European calls and negative Delta European puts.

In [6]:
df_call_25 = BSprice_n_Greeks(S0=S0, K=S0, mu=r, sigma=0.25, 
                            T=T, t=t, opt='call', yield_rate=0) # sigma = 25%, opt = 'call'
df_put_25 = BSprice_n_Greeks(S0=S0, K=S0, mu=r, sigma=0.25, 
                            T=T, t=t, opt='put', yield_rate=0) # sigma = 25%, opt = 'put'

In [7]:
df_call_25

,price_call,delta_call,vega_call
0,5.5984,0.564544,0.196855


In [8]:
df_put_25

,price_put,delta_put,vega_put
0,4.35618,-0.435456,0.196855


In [23]:
vega_callprice_change = (df_call_25.price_call - df_call.price_call) / (0.25 - 0.2) / 100
vega_putprice_change = (df_put_25.price_put - df_put.price_put) / (0.25 - 0.2) / 100
print(f"Change call price respect to change of volatility: {callprice_change[0]:.2f}")
print(f"Change put price respect to change of volatility: {putprice_change[0]:.2f}")

Change call price respect to change of volatility: 0.20
Change put price respect to change of volatility: 0.20


European Call Option
* New price with 25% volatility : 5.60
* Increase in Price (Vega for European Call) : 0.20

European Put Option
* New price with 25% volatility : 4.36
* Increase in Price(Vega for European Put) : 0.20

Both the European call and put option prices increased with the rise in volatility. The increase was approximately **0.20** for both options for each increasing percentage. This indicates that both call and put options become more valuable as the uncertainty or volatility in the underlying asset increases.

**Vega**, the sensitivity of option prices to volatility, is always positive for both call and put options. This is because increased volatility leads to greater uncertainty and a higher chance of the option ending in-the-money, which in turn increases the option's value.

The impact of the increase in volatility was similar for both the call and put options. This is often the case for at-the-money options. Both types of options benefit from increased volatility because it increases the probability of significant price movemeents, either up or down, which can be advantages for option holders.

But note that while the numerical impact was similar, the context differs. For the call option, higher volatility increases the potential for upside movements, making the right to buy more valuable. For the put option, higher volatility amplifies the potential for downside movements, making the right to sell more valuable.

# **STEP2**

In step 2, you will still work with the input data from questions from 5, 6, and 7 from GWP#1, in this case for American derivatives. Specifically  
4. **Team member A** will use Monte-Carlo methods with regular GBM process and daily simulations on an American Call option. Remember to answer the different questions in the original GWP#1:  
price (Q5),  
calculate delta (Q6) and  
vega (Q7) only for the Call option case.

In [70]:
def bs_call_mc(S, K, r, sigma, T, t, opt, Ite):
    data = np.zeros((Ite, 51))
    z = np.random.normal(0, 1, [1, Ite]) # days = 50
    ST = S * np.exp((T-t) * (r - 0.5 * sigma**2) + sigma * np.sqrt(T-t) * z)
    data[:, 1:] = ST - K

    average = np.amax(data, axis=0) / float(Ite)

    callprice =  np.exp(-r * (T-t)) * average
    # delta = 
    # vega = 

    if opt == "call":
        return data, average, callprice

    elif opt == "put":
        return K * np.exp(-r*(T-t)) + callprice - S

In [76]:
S0 = 100
K = S0 # ATM
r = 0.05
sigma = 0.2
T = 3/12
t = 0
Ite = 5000

In [77]:
data, average, call  = bs_call_mc(S=S0, K=K, r=r, sigma=sigma, T=T, t=t, opt='call', Ite=Ite)

In [78]:
call

array([0.        , 0.00966138, 0.00781545, 0.01015368, 0.00911776,
       0.00880577, 0.00913453, 0.0094291 , 0.01077764, 0.01007339,
       0.00844256, 0.01004819, 0.00805141, 0.0102998 , 0.01046196,
       0.00920029, 0.00876428, 0.00842725, 0.00873523, 0.00809646,
       0.00908571, 0.00791246, 0.00777275, 0.00925416, 0.00929879,
       0.0080759 , 0.00875613, 0.00816792, 0.0085401 , 0.00956829,
       0.00966388, 0.00940591, 0.01032608, 0.00868943, 0.00835854,
       0.01047964, 0.00824071, 0.01016785, 0.00809255, 0.00800859,
       0.0084354 , 0.00972287, 0.01126458, 0.00865912, 0.00891353,
       0.00857247, 0.00883402, 0.00871724, 0.00887862, 0.00918304,
       0.00828703])

# **STEP3**

**Team member A** will work with European options with same characteristics as GWP#1 under different levels of moneyness:  
a. Price an European Call option with 110% moneyness and an European Put with 95% moneyness using Black-Scholes. Both have 3 months maturity.  

In [40]:
S0 = 100
K = 90 # 110% moneyness
r = 0.05
sigma = 0.2
T = 3/12
t = 0
opt = 'call'

BSprice_n_Greeks(S0=S0, K=K, mu=r, sigma=sigma, T=T, t=t, opt=opt)

,price_call,delta_call,vega_call
0,11.670087,0.89039,0.093778


In [41]:
S0 = 100
K = 105 # 95% moneyness
r = 0.05
sigma = 0.2
T = 3/12
t = 0
opt = 'put'

BSprice_n_Greeks(S0=S0, K=K, mu=r, sigma=sigma, T=T, t=t, opt=opt)

,price_put,delta_put,vega_put
0,6.173571,-0.622822,0.189941


b. You build a portfolio that buys the previous Call and Put options. What is the delta of the portfolio? How would you delta-hedge this portfolio?  

c. You build a second portfolio that buys the previous Call option and sells the Put. What is the delta of the portfolio? How would you delta-hedge this portfolio?